# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import math

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
import re
import os
import glob

In [2]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
TAXI_ZONES = "taxi_zones/taxi_zones.shp"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber/uber_rides_sample.csv"
WEATHER_PATH = "weather"


NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [3]:
def calculate_distance(from_coord, to_coord):
    # convert longitude and latitude to radian
    lon1, lat1, lon2, lat2 = map(math.radians, from_coord+to_coord) 
    dlon=lon2-lon1
    dlat=lat2-lat1
    a=math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    distance=2*math.asin(math.sqrt(a))*6371*1000
    distance=round(distance/1000,3)
    return distance

In [4]:
def add_distance_column(dataframe):
    dataframe['distance'] = dataframe.apply(lambda x: calculate_distance((
        x['pickup_longitude'], x['pickup_latitude']), (x['dropoff_longitude'], x['dropoff_latitude'])), axis=1)
    return dataframe

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [5]:
def find_taxi_parquet_urls(taxi_url):
    response = requests.get(taxi_url)
    html = response.content
    results_page = bs4.BeautifulSoup(html, 'html.parser')
    links = results_page.find_all('a',title=r'Yellow Taxi Trip Records')
    pattern = r'yellow_tripdata_(2009|201[0-4]|2015-0[1-6])'
    urls = [link.get('href') for link in links if re.search(pattern,link.get('href'))]
    return urls

In [6]:
def get_month_taxi_data(taxi_data_url):
    pattern = r'yellow_tripdata_[0-9]{4}-[0-9]{2}'
    name = re.search(pattern,taxi_data_url).group()
    path = os.path.join(os.getcwd(),'taxi',name+'.csv')
    if os.path.exists(path):
        print('read',name)
        df = pd.read_csv(path)
    else:
        print('download',name)
        df = pd.read_parquet(taxi_data_url, engine='pyarrow')
        df.to_csv(path)
    return df

In [10]:
def clean_month_taxi_data(row_taxi_data,taxi_zones):
    taxi_data = pd.DataFrame()
    if 'tpep_pickup_datetime' in row_taxi_data.columns:
        taxi_data['pickup_datetime'] = row_taxi_data['tpep_pickup_datetime']
    elif 'Trip_Pickup_DateTime' in row_taxi_data.columns:
        taxi_data['pickup_datetime'] = row_taxi_data['Trip_Pickup_DateTime']
    elif 'pickup_datetime' in row_taxi_data.columns:
        taxi_data['pickup_datetime'] = row_taxi_data['pickup_datetime']
    else:
        raise ValueError('No pickup_datetime')
    if 'PULocationID' and 'DOLocationID' in row_taxi_data.columns:
        taxi_data['PULocationID'] = row_taxi_data['PULocationID']
        taxi_data['DOLocationID'] = row_taxi_data['DOLocationID']
        taxi_data = taxi_data.merge(taxi_zones[['LocationID','lon','lat']].set_index('LocationID'),
                                    left_on='PULocationID', right_on='LocationID')
        taxi_data = taxi_data.rename(columns={'lon': 'pickup_longitude', 'lat': 'pickup_latitude'})
        taxi_data = taxi_data.merge(taxi_zones[['LocationID','lon','lat']].set_index('LocationID'),
                                    left_on='DOLocationID', right_on='LocationID')
        taxi_data = taxi_data.rename(columns={'lon': 'dropoff_longitude', 'lat': 'dropoff_latitude'})

        taxi_data = taxi_data.drop(columns=['PULocationID','DOLocationID'])
    elif 'Start_Lon' and 'Start_Lat' and 'End_Lon' and 'End_Lat' in row_taxi_data.columns:
        taxi_data['pickup_longitude'] = row_taxi_data['Start_Lon']
        taxi_data['pickup_latitude'] = row_taxi_data['Start_Lat']
        taxi_data['dropoff_longitude'] = row_taxi_data['End_Lon']
        taxi_data['dropoff_latitude'] = row_taxi_data['End_Lat']
    elif 'pickup_longitude' and 'pickup_latitude' and 'dropoff_longitude' and 'dropoff_latitude' in row_taxi_data.columns:
        taxi_data['pickup_longitude'] = row_taxi_data['pickup_longitude']
        taxi_data['pickup_latitude'] = row_taxi_data['pickup_latitude']
        taxi_data['dropoff_longitude'] = row_taxi_data['dropoff_longitude']
        taxi_data['dropoff_latitude'] = row_taxi_data['dropoff_latitude']
    else:
        raise ValueError('No pickup_datetime')

    taxi_data['pickup_datetime'] = pd.to_datetime(taxi_data['pickup_datetime'])
    taxi_data = taxi_data.dropna()
    
    
    #pick up in bounding box 
    taxi_data = taxi_data[(taxi_data['pickup_longitude']>NEW_YORK_BOX_COORDS[0][1])
                          & (taxi_data['pickup_longitude']<NEW_YORK_BOX_COORDS[1][1])]
    taxi_data = taxi_data[(taxi_data['pickup_latitude']>NEW_YORK_BOX_COORDS[0][0])
                          & (taxi_data['pickup_latitude']<NEW_YORK_BOX_COORDS[1][0])]
    #drop off in bounding box  
    taxi_data = taxi_data[(taxi_data['dropoff_longitude']>NEW_YORK_BOX_COORDS[0][1])
                          & (taxi_data['dropoff_longitude']<NEW_YORK_BOX_COORDS[1][1])]
    taxi_data = taxi_data[(taxi_data['dropoff_latitude']>NEW_YORK_BOX_COORDS[0][0])
                          & (taxi_data['dropoff_latitude']<NEW_YORK_BOX_COORDS[1][0])]
    
    return taxi_data

In [17]:
def get_and_clean_taxi_data():
    all_csv_urls = find_taxi_parquet_urls(TAXI_URL)
    taxi_zone_df = gpd.read_file(TAXI_ZONES)
    taxi_zone_df = taxi_zone_df.to_crs(4326)
    taxi_zone_df['lon'] = taxi_zone_df.centroid.x  
    taxi_zone_df['lat'] = taxi_zone_df.centroid.y
    all_taxi_dataframes = []
    for csv_url in all_csv_urls:
        month_taxi_dataframe = get_month_taxi_data(csv_url)
        month_taxi_dataframe = clean_month_taxi_data(month_taxi_dataframe,taxi_zone_df)
        month_taxi_dataframe = add_distance_column(month_taxi_dataframe)
        all_taxi_dataframes.append(month_taxi_dataframe.sample(30000,random_state=1))
        
    # create one gigantic dataframe with data from every month needed
    all_taxi_data = pd.concat(all_taxi_dataframes)
    all_taxi_data = all_taxi_data.sample(200000,random_state=1)
    return all_taxi_data

In [18]:
taxi_data = get_and_clean_taxi_data()
taxi_data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12324\1040642967.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone_df['lon'] = taxi_zone_df.centroid.x
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12324\1040642967.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone_df['lat'] = taxi_zone_df.centroid.y


read yellow_tripdata_2015-01
read yellow_tripdata_2015-02
read yellow_tripdata_2015-03
read yellow_tripdata_2015-04
read yellow_tripdata_2015-05
read yellow_tripdata_2015-06
read yellow_tripdata_2014-01
read yellow_tripdata_2014-02
read yellow_tripdata_2014-03
read yellow_tripdata_2014-04
read yellow_tripdata_2014-05
read yellow_tripdata_2014-06
read yellow_tripdata_2014-07
read yellow_tripdata_2014-08
read yellow_tripdata_2014-09
read yellow_tripdata_2014-10
read yellow_tripdata_2014-11
read yellow_tripdata_2014-12
read yellow_tripdata_2013-01
read yellow_tripdata_2013-02
read yellow_tripdata_2013-03
read yellow_tripdata_2013-04
read yellow_tripdata_2013-05
read yellow_tripdata_2013-06
read yellow_tripdata_2013-07
read yellow_tripdata_2013-08
read yellow_tripdata_2013-09
read yellow_tripdata_2013-10
read yellow_tripdata_2013-11
read yellow_tripdata_2013-12
read yellow_tripdata_2012-01
read yellow_tripdata_2012-02
read yellow_tripdata_2012-03
read yellow_tripdata_2012-04
read yellow_tr

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2015-01-12 23:07:25,-73.977698,40.758028,-73.981532,40.773633,1.765
1,2015-01-28 20:01:08,-73.977698,40.758028,-73.981532,40.773633,1.765
2,2015-01-27 19:37:57,-73.977698,40.758028,-73.981532,40.773633,1.765
3,2015-01-07 19:02:57,-73.984196,40.759818,-73.981532,40.773633,1.553
4,2015-01-16 13:45:53,-73.984196,40.759818,-73.981532,40.773633,1.553
...,...,...,...,...,...,...
2595,2009-12-10 11:30:34,-73.991986,40.749751,-73.996724,40.742683,0.881
2596,2009-12-27 00:01:49,-73.982310,40.772300,-73.964232,40.809630,4.421
2597,2009-12-06 16:18:46,-73.993775,40.751384,-73.993252,40.752434,0.125
2598,2009-12-12 13:03:00,-73.969870,40.757927,-74.001047,40.722898,4.698


### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [19]:
def clean_uber_data(csv_file):
    uber_data = pd.read_csv(csv_file)
    uber_data = uber_data.drop(columns=['Unnamed: 0','key','fare_amount','passenger_count'])
    uber_data = uber_data.dropna()
    
    #pick up in bounding box 
    uber_data = uber_data[(uber_data['pickup_longitude']>NEW_YORK_BOX_COORDS[0][1]) & (uber_data['pickup_longitude']<NEW_YORK_BOX_COORDS[1][1])]
    uber_data = uber_data[(uber_data['pickup_latitude']>NEW_YORK_BOX_COORDS[0][0]) & (uber_data['pickup_latitude']<NEW_YORK_BOX_COORDS[1][0])]
    #drop off in bounding box  
    uber_data = uber_data[(uber_data['dropoff_longitude']>NEW_YORK_BOX_COORDS[0][1]) & (uber_data['dropoff_longitude']<NEW_YORK_BOX_COORDS[1][1])]
    uber_data = uber_data[(uber_data['dropoff_latitude']>NEW_YORK_BOX_COORDS[0][0]) & (uber_data['dropoff_latitude']<NEW_YORK_BOX_COORDS[1][0])]
    uber_data['pickup_datetime'] = pd.to_datetime(uber_data['pickup_datetime'])
    
    return uber_data

In [20]:
def get_uber_data():
    uber_dataframe = clean_uber_data(UBER_CSV)
    uber_dataframe = add_distance_column(uber_dataframe)
    return uber_dataframe

In [21]:
uber_data = get_uber_data()
uber_data

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1.683
1,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,2.458
2,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,5.036
3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,1.662
4,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,4.475
...,...,...,...,...,...,...
199995,2012-10-28 10:49:00+00:00,-73.987042,40.739367,-73.986525,40.740297,0.112
199996,2014-03-14 01:09:00+00:00,-73.984722,40.736837,-74.006672,40.739620,1.875
199997,2009-06-29 00:42:00+00:00,-73.986017,40.756487,-73.858957,40.692588,12.850
199998,2015-05-20 14:56:25+00:00,-73.997124,40.725452,-73.983215,40.695415,3.540


### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [22]:
def clean_month_weather_data_hourly(csv_file):
    whole_data = pd.read_csv(csv_file)
    whole_data['REPORT_TYPE'] = whole_data['REPORT_TYPE'].astype(str)
    whole_data['DATE'] = pd.to_datetime(whole_data['DATE'])
    
    hourly_data = whole_data[whole_data['REPORT_TYPE'] != 'SOD  ']
    hourly_data = hourly_data[['DATE','HourlyPrecipitation','HourlyWindSpeed']]
    hourly_data['HourlyPrecipitation'] = pd.to_numeric(hourly_data['HourlyPrecipitation'], errors='coerce')
    hourly_data['HourlyWindSpeed'] = pd.to_numeric(hourly_data['HourlyWindSpeed'], errors='coerce')
    hourly_data['HourlyPrecipitation'].fillna(0, inplace=True)
    hourly_data['HourlyWindSpeed'].fillna(0, inplace=True)
    hourly_data = hourly_data.rename(columns={'DATE': 'datetime'})
    hourly_data = hourly_data.rename(columns={'HourlyPrecipitation':'hourly_precipitation','HourlyWindSpeed':'hourly_wind_speed'})
    
    return hourly_data

In [23]:
def clean_month_weather_data_daily(csv_file):
    whole_data = pd.read_csv(csv_file)
    whole_data['REPORT_TYPE'] = whole_data['REPORT_TYPE'].astype(str)
    whole_data['DATE'] = pd.to_datetime(whole_data['DATE'])
    whole_data['DATE'] = whole_data['DATE'].dt.date
    
    hourly_data = whole_data[whole_data['REPORT_TYPE'] != 'SOD  ']
    hourly_data = hourly_data[['DATE','HourlyPrecipitation','HourlyWindSpeed']]
    hourly_data['HourlyPrecipitation'] = pd.to_numeric(hourly_data['HourlyPrecipitation'], errors='coerce')
    hourly_data['HourlyWindSpeed'] = pd.to_numeric(hourly_data['HourlyWindSpeed'], errors='coerce')
    hourly_data['HourlyPrecipitation'].fillna(0, inplace=True)
    hourly_data = hourly_data.groupby('DATE', as_index=False).agg({'HourlyWindSpeed': 'mean', 'HourlyPrecipitation': 'sum'})
    
    daily_data = whole_data[whole_data['REPORT_TYPE'] == 'SOD  ']
    daily_data = daily_data[['DATE','Sunrise','Sunset']]
    daily_data['Sunrise'] = pd.to_numeric(daily_data['Sunrise'], errors='coerce')
    daily_data['Sunset'] = pd.to_numeric(daily_data['Sunset'], errors='coerce')
    
    daily_data = hourly_data.merge(daily_data, on='DATE',how='left')
    daily_data.fillna(method='ffill',inplace=True)
    daily_data.fillna(method='bfill',inplace=True)
    daily_data = daily_data.rename(columns={'DATE': 'date'})
    daily_data = daily_data.rename(columns={'HourlyPrecipitation':'daily_precipitation', 'HourlyWindSpeed':'daily_wind_speed'})
    daily_data = daily_data.rename(columns={'Sunrise': 'sunrise', 'Sunset': 'sunset'})
    
    return daily_data

In [24]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    weather_csv_files = glob.glob(os.path.join(os.getcwd(), WEATHER_PATH, "*.csv"))
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [25]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()
hourly_weather_data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12324\156112435.py:2: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  whole_data = pd.read_csv(csv_file)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12324\2991833665.py:2: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  whole_data = pd.read_csv(csv_file)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12324\156112435.py:2: DtypeWarning: Columns (8,9,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  whole_data = pd.read_csv(csv_file)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12324\2991833665.py:2: DtypeWarning: Columns (8,9,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  whole_data = pd.read_csv(csv_file)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12324\156112435.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory

,datetime,hourly_precipitation,hourly_wind_speed
0,2009-01-01 00:51:00,0.0,18.0
1,2009-01-01 01:51:00,0.0,18.0
2,2009-01-01 02:51:00,0.0,18.0
3,2009-01-01 03:51:00,0.0,8.0
4,2009-01-01 04:51:00,0.0,11.0
...,...,...,...
11381,2015-12-31 20:51:00,0.0,10.0
11382,2015-12-31 21:51:00,0.0,0.0
11383,2015-12-31 22:51:00,0.0,7.0
11384,2015-12-31 23:51:00,0.0,5.0


In [26]:
daily_weather_data

,date,daily_wind_speed,daily_precipitation,sunrise,sunset
0,2009-01-01,11.041667,0.00,720.0,1640.0
1,2009-01-02,6.806452,0.00,720.0,1640.0
2,2009-01-03,9.875000,0.00,720.0,1640.0
3,2009-01-04,7.370370,0.00,720.0,1640.0
4,2009-01-05,6.925926,0.00,720.0,1640.0
...,...,...,...,...,...
360,2015-12-27,4.911111,0.17,719.0,1635.0
361,2015-12-28,8.208333,0.03,719.0,1636.0
362,2015-12-29,7.787234,0.93,720.0,1636.0
363,2015-12-30,4.184211,0.29,720.0,1637.0


In [27]:
taxi_data

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2015-01-12 23:07:25,-73.977698,40.758028,-73.981532,40.773633,1.765
1,2015-01-28 20:01:08,-73.977698,40.758028,-73.981532,40.773633,1.765
2,2015-01-27 19:37:57,-73.977698,40.758028,-73.981532,40.773633,1.765
3,2015-01-07 19:02:57,-73.984196,40.759818,-73.981532,40.773633,1.553
4,2015-01-16 13:45:53,-73.984196,40.759818,-73.981532,40.773633,1.553
...,...,...,...,...,...,...
2595,2009-12-10 11:30:34,-73.991986,40.749751,-73.996724,40.742683,0.881
2596,2009-12-27 00:01:49,-73.982310,40.772300,-73.964232,40.809630,4.421
2597,2009-12-06 16:18:46,-73.993775,40.751384,-73.993252,40.752434,0.125
2598,2009-12-12 13:03:00,-73.969870,40.757927,-74.001047,40.722898,4.698


In [28]:
uber_data

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1.683
1,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,2.458
2,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,5.036
3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,1.662
4,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,4.475
...,...,...,...,...,...,...
199995,2012-10-28 10:49:00+00:00,-73.987042,40.739367,-73.986525,40.740297,0.112
199996,2014-03-14 01:09:00+00:00,-73.984722,40.736837,-74.006672,40.739620,1.875
199997,2009-06-29 00:42:00+00:00,-73.986017,40.756487,-73.858957,40.692588,12.850
199998,2015-05-20 14:56:25+00:00,-73.997124,40.725452,-73.983215,40.695415,3.540


### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [29]:
engine = db.create_engine(DATABASE_URL)

In [30]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    datetime DATETIME,
    hourly_precipitation FLOAT,
    hourly_wind_speed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    date DATE,
    daily_wind_speed FLOAT,
    daily_precipitation FLOAT,
    sunrise INT32,
    sunset INT32
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
);
"""

In [31]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [32]:
# create the tables with the schema files
with engine.connect() as connection:
    sql = open(DATABASE_SCHEMA_FILE, "r")
    commands = sql.read().split(";")
    sql.close()
    for command in commands:
        connection.execute(command)

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [33]:
def write_dataframes_to_table(table_to_df_dict):
    for name, table in table_to_df_dict.items():
        table.to_sql(name, engine, if_exists='replace', index=False)

In [34]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

In [35]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [36]:
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.

In [37]:
QUERY_1 = """
SELECT strftime('%H', pickup_datetime) AS time, COUNT(*) AS number
FROM taxi_trips
GROUP BY time
ORDER BY number DESC;
"""

In [38]:
engine.execute(QUERY_1).fetchall()

[('19', 12476),
 ('18', 11819),
 ('20', 11558),
 ('21', 11502),
 ('22', 10985),
 ('14', 10249),
 ('13', 9808),
 ('23', 9772),
 ('12', 9670),
 ('17', 9623),
 ('15', 9484),
 ('11', 9248),
 ('09', 9026),
 ('08', 8997),
 ('10', 8994),
 ('16', 8242),
 ('00', 7836),
 ('07', 7023),
 ('01', 5775),
 ('02', 4356),
 ('06', 4032),
 ('03', 3199),
 ('04', 2282),
 ('05', 1943)]

In [39]:
write_query_to_file(QUERY_1, "most_popular_hour.sql")

In [40]:
QUERY_2 = """
SELECT strftime('%w', pickup_datetime) AS day, COUNT(*) AS number
FROM uber_trips
GROUP BY day
ORDER BY number DESC;
"""

In [41]:
engine.execute(QUERY_2).fetchall()

[('5', 30166),
 ('6', 29599),
 ('4', 29338),
 ('3', 28328),
 ('2', 27526),
 ('0', 25834),
 ('1', 24681)]

In [42]:
write_query_to_file(QUERY_2, "most_popular_day.sql")

In [43]:
QUERY_3 = """
WITH hired_trips AS
(
    SELECT pickup_datetime, distance FROM taxi_trips 
    WHERE pickup_datetime BETWEEN '2013-07-01' AND '2013-08-01'
    UNION ALL
    SELECT pickup_datetime, distance FROM uber_trips
    WHERE pickup_datetime BETWEEN '2013-07-01' AND '2013-08-01'
)
SELECT distance
FROM hired_trips
ORDER BY distance
LIMIT 1
OFFSET (SELECT COUNT(*) FROM hired_trips) * 95 / 100 - 1 ;
"""

In [44]:
engine.execute(QUERY_3).fetchall()

[(9.948,)]

In [45]:
write_query_to_file(QUERY_3, "95%_percentile_distance.sql")

In [46]:
QUERY_4 = """
WITH hired_trips AS
(
    SELECT pickup_datetime, distance FROM taxi_trips 
    WHERE pickup_datetime BETWEEN '2009-01-01' AND '2010-01-01'
    UNION ALL
    SELECT pickup_datetime, distance FROM uber_trips
    WHERE pickup_datetime BETWEEN '2009-01-01' AND '2010-01-01'
)
SELECT DATE(pickup_datetime) AS date, COUNT(*) AS number, printf("%.4f",AVG(distance)) AS avg_distance
FROM hired_trips
GROUP BY date
ORDER BY number DESC
LIMIT 10;
"""

In [47]:
engine.execute(QUERY_4).fetchall()

[('2009-12-11', 251, '2.9105'),
 ('2009-06-05', 226, '2.7465'),
 ('2009-08-14', 223, '3.1896'),
 ('2009-02-20', 219, '2.7915'),
 ('2009-05-16', 218, '2.9435'),
 ('2009-12-05', 215, '3.1627'),
 ('2009-10-23', 214, '2.8785'),
 ('2009-05-21', 213, '3.3273'),
 ('2009-03-12', 211, '2.9953'),
 ('2009-11-05', 209, '3.1431')]

In [48]:
write_query_to_file(QUERY_4, "top_10_days.sql")

In [49]:
QUERY_5 = """
WITH hired_trips AS
(
    SELECT DATE(pickup_datetime) AS date FROM taxi_trips
    WHERE date BETWEEN '2014-01-01' AND '2015-01-01'
    UNION ALL
    SELECT DATE(pickup_datetime) AS date FROM uber_trips
    WHERE date BETWEEN '2014-01-01' AND '2015-01-01'
)
SELECT hired_trips.date as date, printf("%.4f",daily_wind_speed), COUNT(*) AS number
FROM hired_trips
JOIN daily_weather ON hired_trips.date = DATE(daily_weather.date)
GROUP BY hired_trips.date
ORDER BY daily_wind_speed DESC
LIMIT 10
"""

In [50]:
engine.execute(QUERY_5).fetchall()

[('2014-03-13', '14.0000', 198),
 ('2014-01-07', '13.0833', 175),
 ('2014-01-02', '12.7273', 128),
 ('2014-02-13', '12.2264', 139),
 ('2014-03-26', '11.9545', 172),
 ('2014-03-29', '11.9149', 195),
 ('2014-12-07', '11.6000', 149),
 ('2014-12-09', '11.2692', 168),
 ('2014-12-08', '11.2667', 156),
 ('2014-11-02', '10.8261', 147)]

In [51]:
write_query_to_file(QUERY_5, "windiest_10_days.sql")

In [52]:
QUERY_6 = """
WITH hired_trips AS
(
    SELECT strftime('%Y-%m-%d %H',pickup_datetime) AS trip_hour FROM taxi_trips
    WHERE pickup_datetime BETWEEN '2012-10-22' AND '2012-10-31'
    UNION ALL
    SELECT strftime('%Y-%m-%d %H',pickup_datetime) AS trip_hour FROM uber_trips
    WHERE pickup_datetime BETWEEN '2012-10-22' AND '2012-10-31'
),
hurricane_weather AS
(
    SELECT strftime('%Y-%m-%d %H',datetime) AS weather_hour, hourly_precipitation, hourly_wind_speed FROM hourly_weather
    WHERE datetime BETWEEN '2012-10-22' AND '2012-10-31'
)
SELECT weather_hour, COALESCE(COUNT(trip_hour),0) AS number, hourly_precipitation, hourly_wind_speed
FROM hurricane_weather
LEFT JOIN hired_trips
ON trip_hour = weather_hour
GROUP BY weather_hour
"""

In [53]:
engine.execute(QUERY_6).fetchall()

[('2012-10-22 00', 4, 0.0, 7.0),
 ('2012-10-22 01', 0, 0.0, 5.0),
 ('2012-10-22 02', 2, 0.0, 7.0),
 ('2012-10-22 03', 1, 0.0, 0.0),
 ('2012-10-22 04', 0, 0.0, 0.0),
 ('2012-10-22 05', 3, 0.0, 0.0),
 ('2012-10-22 06', 8, 0.0, 5.0),
 ('2012-10-22 07', 9, 0.0, 3.0),
 ('2012-10-22 08', 4, 0.0, 3.0),
 ('2012-10-22 09', 12, 0.0, 5.0),
 ('2012-10-22 10', 6, 0.0, 0.0),
 ('2012-10-22 11', 10, 0.0, 0.0),
 ('2012-10-22 12', 8, 0.0, 11.0),
 ('2012-10-22 13', 11, 0.0, 0.0),
 ('2012-10-22 14', 5, 0.0, 7.0),
 ('2012-10-22 15', 5, 0.0, 6.0),
 ('2012-10-22 16', 8, 0.0, 3.0),
 ('2012-10-22 17', 9, 0.0, 7.0),
 ('2012-10-22 18', 12, 0.0, 5.0),
 ('2012-10-22 19', 11, 0.0, 5.0),
 ('2012-10-22 20', 7, 0.0, 3.0),
 ('2012-10-22 21', 7, 0.0, 0.0),
 ('2012-10-22 22', 11, 0.0, 3.0),
 ('2012-10-22 23', 1, 0.0, 3.0),
 ('2012-10-23 00', 5, 0.0, 3.0),
 ('2012-10-23 01', 2, 0.0, 0.0),
 ('2012-10-23 02', 1, 0.0, 3.0),
 ('2012-10-23 03', 0, 0.0, 0.0),
 ('2012-10-23 04', 0, 0.0, 3.0),
 ('2012-10-23 05', 2, 0.0, 0.0),
 ('

In [54]:
write_query_to_file(QUERY_6, "hurricane_sandy.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)